## Registar um pipeline no mlflow

In [12]:
import mlflow
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.metrics import  precision_recall_curve, roc_auc_score, confusion_matrix, accuracy_score, recall_score, precision_score, f1_score,auc, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn import tree
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier
from mlflow.models import infer_signature



In [13]:
root_path = '../data/'
seed = 42
target_col = "default.payment.next.month"

## Definir a diretoria onde as experiências são guardadas

In [14]:
from pathlib import Path

uri = "http://127.0.0.1:5000"

mlflow.set_tracking_uri(uri)

## Fazer set da experiência

In [15]:
mlflow.set_experiment("Lending Prediction Experiment")

2025/03/27 00:25:07 INFO mlflow.tracking.fluent: Experiment with name 'Lending Prediction Experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/226254833090476706', creation_time=1743035107482, experiment_id='226254833090476706', last_update_time=1743035107482, lifecycle_stage='active', name='Lending Prediction Experiment', tags={}>

## Criar os datasets


In [16]:
file_path = root_path + 'lending_data.csv'
df = pd.read_csv(file_path)

df = df.drop('ID', axis = 1)

train_set, test_set = train_test_split(df, test_size = 0.2, random_state = seed)

train_set_staged = train_set.sample(frac=0.2)

X_train = train_set_staged.drop([target_col], axis = 'columns')
y_train = train_set_staged[target_col]

X_test = test_set.drop([target_col], axis = 1)
y_test = test_set[target_col]

X_train.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
15887,360000.0,2,1,2,27,0,0,0,0,0,...,110795.0,101668.0,78730.0,66682.0,4500.0,4100.0,4208.0,3000.0,2300.0,1800.0
29449,280000.0,1,1,1,31,-2,-2,-2,-2,-2,...,1345.0,-5.0,-5.0,985.0,0.0,1351.0,0.0,0.0,990.0,3090.0
5313,80000.0,2,1,1,50,1,-2,-2,-2,-2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16926,90000.0,1,3,2,50,0,0,0,0,0,...,71616.0,60177.0,54440.0,45788.0,3451.0,3243.0,2390.0,1840.0,1690.0,1954.0
22133,80000.0,2,1,2,22,1,2,2,-1,-1,...,4258.0,7032.0,6348.0,1606.0,1000.0,0.0,7032.0,6348.0,1606.0,0.0


In [17]:
# Definição dos modelos e dos seus respectivos hiper-parâmetros para busca em grid
models_and_params = [
    (
        'ann',
        Pipeline(steps=[
            ('scaler', MinMaxScaler()),
            ('Classifier', MLPClassifier(solver = 'lbfgs',  random_state = seed, max_iter = 1000))
        ]),
        {
          "Classifier__hidden_layer_sizes": [(20,), (20,10), (20, 10, 2)],
          'Classifier__learning_rate_init':[0.0001, 0.001, 0.01, 0.1]
        }
    ),
    (
        'random_forest',
        RandomForestClassifier(random_state = seed,  class_weight = 'balanced'),
        {
            'n_estimators':[10, 100, 300, 1000]
        }
    ),
    (
        'decision_tree',
        tree.DecisionTreeClassifier(random_state = seed,  class_weight = 'balanced'),
        {
            'max_depth':[3, 6],
            'min_samples_split': [2, 4, 10]
        }
    ),
    (
        'svm',
        Pipeline(steps=[
            ('scaler', MinMaxScaler()),
            ('Classifier', SVC(random_state = seed, class_weight = 'balanced', gamma = 'scale', probability = True, verbose = True))
        ]),
        {
          "Classifier__C": [0.1, 1, 10],
          "Classifier__kernel": ["linear","rbf"]
        }
    ),
    (
        'knn',
        Pipeline(steps=[
            ('scaler', MinMaxScaler()),
            ('Classifier', KNeighborsClassifier())
        ]),
        {
          "Classifier__n_neighbors": [1,10,100],
        }
    ),
    (
        'lr',
        Pipeline(steps=[
            ('scaler', MinMaxScaler()),
            ('Classifier', LogisticRegression(max_iter = 500, solver = 'lbfgs', random_state = seed, class_weight = 'balanced'))
        ]),
        {
          "Classifier__C": [0.001, 0.01, 0.1, 1, 10, 100]
        }
    )
]

In [18]:
for model_str, clf, params in models_and_params:
    print(f"Currently assessing {model_str}")
    
        # Inicia a run
    with mlflow.start_run(run_name=model_str):

        
        # 1️⃣ Executar Grid Search
        grid_search = GridSearchCV(clf, params, cv=5, n_jobs=-1)
        grid_search.fit(X_train, y_train)

        # 2️⃣ Obter os melhores parâmetros e o melhor score
        best_params = grid_search.best_params_
        best_score = grid_search.best_score_

        # 3️⃣ Registrar os hiperparâmetros e a métrica no MLflow
        mlflow.log_params(best_params)
        mlflow.log_metric("best_cv_score", best_score)

        # 4️⃣ Salvar o modelo treinado no MLflow
        mlflow.sklearn.log_model(grid_search.best_estimator_, artifact_path="sklearn-model", input_example=X_train, registered_model_name=model_str)

        # 5️⃣ Fazer predições no conjunto de teste
        y_preds = grid_search.best_estimator_.predict(X_test)

        # 6️⃣ Calcular métricas adicionais
        acc = accuracy_score(y_test, y_preds)
        mlflow.log_metric("accuracy", acc)

        print(f"✅ Model {model_str} completed with best score: {best_score:.4f}")


print("🎯 All models have been evaluated and logged in MLflow!")

Currently assessing ann


c:\Users\a79918\AppData\Local\miniconda3\envs\OML_rumos_bank\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
c:\Users\a79918\AppData\Local\miniconda3\envs\OML_rumos_bank\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer colum

✅ Model ann completed with best score: 0.7881
🏃 View run ann at: http://127.0.0.1:5000/#/experiments/226254833090476706/runs/59d0339a24a240cd8ff94eaa2135af23
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/226254833090476706
Currently assessing random_forest


c:\Users\a79918\AppData\Local\miniconda3\envs\OML_rumos_bank\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
c:\Users\a79918\AppData\Local\miniconda3\envs\OML_rumos_bank\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represe

✅ Model random_forest completed with best score: 0.8042
🏃 View run random_forest at: http://127.0.0.1:5000/#/experiments/226254833090476706/runs/814867d913514bdb948def4bc48307c5
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/226254833090476706
Currently assessing decision_tree


...

c:\Users\a79918\AppData\Local\miniconda3\envs\OML_rumos_bank\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
c:\Users\a79918\AppData\Local\miniconda3\envs\OML_rumos_bank\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represe

✅ Model knn completed with best score: 0.7944
🏃 View run knn at: http://127.0.0.1:5000/#/experiments/226254833090476706/runs/43c45f2aff3140bf9d3bc59e62959214
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/226254833090476706
Currently assessing lr


c:\Users\a79918\AppData\Local\miniconda3\envs\OML_rumos_bank\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
c:\Users\a79918\AppData\Local\miniconda3\envs\OML_rumos_bank\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represe

✅ Model lr completed with best score: 0.6800
🏃 View run lr at: http://127.0.0.1:5000/#/experiments/226254833090476706/runs/c83d4524ab0041bbac44e107e2c20be1
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/226254833090476706
🎯 All models have been evaluated and logged in MLflow!


Created version '1' of model 'lr'.
